# Project 2
Authors: Richie Rivera and Naomi Buell

*Instructions*
1. *Identify a large 2-node network dataset—you can start with a dataset in a repository.  Your data should meet the criteria that it consists of ties between and not within two (or more) distinct groups.*
2. *Reduce the size of the network using a method such as the island method described in chapter 4 of social network analysis.*
3. *What can you infer about each of the distinct groups?*

## 1. Identify and load data

In [6]:
# Import libraries
import networkx as nx
import pandas as pd
import numpy as np
import requests
import zipfile
import io

In [ ]:
# Download the zip file
zip_url = 'https://github.com/KS-92/Manga/raw/main/Manga_network_data.zip'
response = requests.get(zip_url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    # Extract the required files to memory
    with z.open('Manga_network_data/Temporal/Temporal_One_Piece.csv') as f:
        temporal_df = pd.read_csv(f)
    with z.open('Manga_network_data/Static/Static_One_Piece.edgelist') as f:
        G_static = nx.read_weighted_edgelist(f, nodetype=int)
        static_edges = nx.to_pandas_edgelist(G_static)

# Show the first few rows of each dataframe
print(temporal_df.head())
print(static_edges.head())

   i  j  t
0  1  2  3
1  1  2  4
2  1  2  5
3  1  2  6
4  1  2  7
   source  target  weight
0       1       2    31.0
1       1       4    16.0
2       1      26    19.0
3       1      33     9.0
4       1      30    13.0


## 2. Perform island method

## 3. Inference

We can infer that...

## References
Sugishita, K. and Masuda, N., Social network analysis of manga: similarities to real-world social networks and trends over decades, Applied Network Science, 8, 79, 2023.